# ai.py notes

Importing the libraries

In [6]:
import numpy as np
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable

Creating the architecture of the Neural Network

In [8]:
class Network(nn.Module):
    
    def __init__(self, input_size, nb_action):
        '''
        input_size = size of input vector
        nb_action = number of actions
        '''
        # inheritance in python
        super(Network, self).__init__()
        
        self.input_size = input_size
        self.nb_action = nb_action
        
        # creating a fully connected layer with linear activation function
        self.fc1 = nn.Linear(in_features=self.input_size, out_features=30, bias=True)
        
        #creating a fully connected layer with linear activation function
        self.fc2 = nn.Linear(in_features=30, out_features=nb_action, bias=True)
        
        
        